In [3]:
%pip install boto3
%pip install polars


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
  Using cached polars-0.15.14-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (14.8 MB)
Note: you may need to restart the kernel to use updated packages.


In [7]:
# 1st party imports
from datetime import date, datetime, timedelta
import gzip

# 3rd party imports
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import pandas as pd
import polars as pl

pd.options.mode.chained_assignment = None



In [8]:
S3_BUCKET = "ooni-data-eu-fra"
S3_PREFIX = "raw"

def build_prefix(
    timestamp: datetime,
    cc: str,
    test: str = "webconnectivity",
) -> str:
    """Build the S3 prefix"""
    yyyy = f"{timestamp.year:04d}"
    mm = f"{timestamp.month:02d}"
    dd = f"{timestamp.day:02d}"
    hh = f"{timestamp.hour:02d}"
    cc = cc.upper()
    test = test.lower()
    # return f"s3://{S3_BUCKET}/{S3_PREFIX}/{yyyy}{mm}{dd}/{hh}/{cc}/{test}/"
    return f"{S3_PREFIX}/{yyyy}{mm}{dd}/{hh}/{cc}/{test}/"



In [9]:

ts = datetime(year=2023, month=1, day=5, hour=10)
s3_prefix = build_prefix(ts, "ru")

aws_config = Config(signature_version=UNSIGNED)
# s3 = boto3.client('s3', config=aws_config)
# objs = s3.list_objects_v2(Bucket=S3_BUCKET, Prefix=s3_prefix)

s3 = boto3.resource('s3', config=aws_config)
s3_bucket = s3.Bucket(S3_BUCKET)
objs = s3_bucket.objects.filter(Prefix=s3_prefix)
objs = filter(lambda o: o.key.endswith(".jsonl.gz"), objs)

o = next(objs)

df = pd.read_json(
    o.get()['Body'],
    lines=True,
    compression='gzip',
)
df = df[0:3]



In [12]:
df

# dfp = pl.from_pandas(df)
# dfp

,annotations,data_format_version,input,measurement_start_time,probe_asn,probe_cc,probe_ip,probe_network_name,report_id,resolver_asn,...,resolver_network_name,software_name,software_version,test_helpers,test_keys,test_name,test_runtime,test_start_time,test_version,extensions
0,"{'architecture': 'arm64', 'engine_name': 'ooni...",0.2.0,https://severrea.github.io/,2023-01-05 09:59:59,AS16143,RU,127.0.0.1,OOO NIIR-RadioNet,20230105T095822Z_webconnectivity_RU_16143_n1_D...,AS15169,...,Google LLC,ooniprobe-android-unattended,3.7.3,{'backend': {'address': 'https://0.th.ooni.org...,"{'agent': 'redirect', 'client_resolver': '172....",web_connectivity,0.453488,2023-01-05 09:58:22,0.4.2,NaN
1,"{'architecture': 'arm64', 'engine_name': 'ooni...",0.2.0,https://shtab.navalny.com/,2023-01-05 10:00:00,AS16143,RU,127.0.0.1,OOO NIIR-RadioNet,20230105T095822Z_webconnectivity_RU_16143_n1_D...,AS15169,...,Google LLC,ooniprobe-android-unattended,3.7.3,{'backend': {'address': 'https://0.th.ooni.org...,"{'agent': 'redirect', 'client_resolver': '172....",web_connectivity,0.258107,2023-01-05 09:58:22,0.4.2,NaN
2,"{'architecture': 'arm64', 'engine_name': 'ooni...",0.2.0,https://sibrea.github.io/,2023-01-05 10:00:00,AS16143,RU,127.0.0.1,OOO NIIR-RadioNet,20230105T095822Z_webconnectivity_RU_16143_n1_D...,AS15169,...,Google LLC,ooniprobe-android-unattended,3.7.3,{'backend': {'address': 'https://0.th.ooni.org...,"{'agent': 'redirect', 'client_resolver': '172....",web_connectivity,0.500912,2023-01-05 09:58:22,0.4.2,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   annotations             3 non-null      object        
 1   data_format_version     3 non-null      object        
 2   input                   3 non-null      object        
 3   measurement_start_time  3 non-null      datetime64[ns]
 4   probe_asn               3 non-null      object        
 5   probe_cc                3 non-null      object        
 6   probe_ip                3 non-null      object        
 7   probe_network_name      3 non-null      object        
 8   report_id               3 non-null      object        
 9   resolver_asn            3 non-null      object        
 10  resolver_ip             3 non-null      object        
 11  resolver_network_name   3 non-null      object        
 12  software_name           3 non-null      object        

In [6]:
########
########
########
########
########
########
######## OBSOLETE
########
########
########
########


_df = df[:1]

config = [
    'annotations.architecture',
    'annotations.engine_name',
    'annotations.engine_version',
    'annotations.flavor',
    'annotations.network_type',
    'annotations.origin',
    'annotations.platform',
    'test_keys.agent',
    'test_keys.client_resolver',
    'test_keys.network_events[].address',
    'test_keys.network_events[].failure',
    'test_keys.network_events[].operation',
    'test_keys.network_events[].proto',
    'test_keys.network_events[].t',
    'test_keys.network_events[].tags',
    'test_keys.queries[].answers',
    'test_keys.queries[].engine',
    'test_keys.queries[].failure',
    'test_keys.queries[].hostname',
    'test_keys.queries[].query_type',
    'test_keys.queries[].resolver_hostname',
    'test_keys.queries[].resolver_port',
    'test_keys.queries[].resolver_address',
    'test_keys.dns_experiment_failure',
    'test_keys.control_failure',
    'test_keys.control',
    'test_keys.tcp_connect[].ip',
    'test_keys.tcp_connect[].port',
    'test_keys.tcp_connect[].status',
    'test_keys.requests[].failure',
    'test_keys.http_experiment_failure',
    'test_keys.body_length_match',
    'test_keys.body_proportion',
    'test_keys.status_code_match',
    'test_keys.headers_match',
    'test_keys.title_match',
    'test_keys.accessible',
    'test_keys.blocking',
]

to_be_dropped = set()


def flatten(df, col, cols):
    if len(cols) == 0:
        return df
    
    _df = df
    
    to_be_dropped = set()
    
    # If the column is a list, then explode before extracting anything further
    if col.endswith("[]"):
        col = col.replace("[]", "")
        _df = _df.explode(col, ignore_index=True)
    
    # Extract all attributes in there.
    for attr in cols:
        _df[f"{col}_{attr}"] = _df.apply(lambda r: r[col].get(attr, None), axis=1)
        
    _df = _df.drop(col, axis=1)
    return flatten(_df, cols[0], cols[1:])
            
        

for k in keys:
    parts = k.split(',')
    l = len(parts)
    
    
    col, attr = k.split(".")
    _df[f'{col}_{attr}'] = _df.apply(lambda r: r[col][attr], axis=1)
    to_be_dropped |= { col }
    
_df = _df.drop(labels=to_be_dropped, axis=1)

cols = [
    'data_format_version',
    'input',
    'measurement_start_time',
    'probe_asn',
    'probe_cc',
    'probe_network_name',
    'resolver_asn',
    'resolver_ip',
    'resolver_network_name',
    'software_name',
    'software_version',
    'annotations_architecture',
    'annotations_engine_name',
    'annotations_engine_version',
    'annotations_flavor',
    'annotations_network_type',
    'annotations_origin',
    'annotations_platform',
    'test_keys_agent',
    'test_keys_client_resolver',
    'test_keys_retries',
    'test_keys_socksproxy',
    'test_keys_network_events', # flatten
    # 'test_keys_tls_handshakes', # flatten
    'test_keys_queries', # flatten
    'test_keys_dns_experiment_failure',
    'test_keys_control_failure',
    # 'test_keys_control',  # dict
    'test_keys_tcp_connect', # flatten
    'test_keys_requests', # flatten
    'test_keys_http_experiment_failure',
    'test_keys_body_length_match',
    'test_keys_body_proportion',
    'test_keys_status_code_match',
    'test_keys_headers_match',
    'test_keys_title_match',
    'test_keys_accessible',
    'test_keys_blocking',
]


_df = _df[cols]

explode = {
    'test_keys_network_events': [ 'address', 'failure', 'operation', 'proto', 't', 'tags'],
    'test_keys_tcp_connect': ['ip', 'port', 'status'],
    'test_keys_queries': ['answers', 'engine', 'failure',' hostname', 'query_type', 'resolver_hostname', 'resolver_port', 'resolver_address'],
    'test_keys_requests': ['failure'],
}

to_be_dropped = set()

for col, attributes in explode.items():
    _df = _df.explode(col, ignore_index=True)

    for attr in attributes:
        # _df[f'{col}_{attr}'] = _df.apply(lambda r: r[col][attr], axis=1)
        _df[f'{col}_{attr}'] = _df.apply(lambda r: r[col].get(attr, None), axis=1)

    to_be_dropped |= { col }

_df = _df.drop(labels=to_be_dropped, axis=1)
    
_df.T

NameError: name 'keys' is not defined

In [7]:
df_test = df[:1]

config = [
    'annotations.architecture',
    'annotations.engine_name',
    'annotations.engine_version',
    'annotations.flavor',
    'annotations.network_type',
    'annotations.origin',
    'annotations.platform',
    'test_keys.agent',
    'test_keys.client_resolver',
    'test_keys.network_events[].address',
    'test_keys.network_events[].failure',
    'test_keys.network_events[].operation',
    'test_keys.network_events[].proto',
    'test_keys.network_events[].t',
    'test_keys.network_events[].tags',
    'test_keys.queries[].answers',
    'test_keys.queries[].engine',
    'test_keys.queries[].failure',
    'test_keys.queries[].hostname',
    'test_keys.queries[].query_type',
    'test_keys.queries[].resolver_hostname',
    'test_keys.queries[].resolver_port',
    'test_keys.queries[].resolver_address',
    'test_keys.dns_experiment_failure',
    'test_keys.control_failure',
    'test_keys.control',
    'test_keys.tcp_connect[].ip',
    'test_keys.tcp_connect[].port',
    'test_keys.tcp_connect[].status',
    'test_keys.requests[].failure',
    'test_keys.http_experiment_failure',
    'test_keys.body_length_match',
    'test_keys.body_proportion',
    'test_keys.status_code_match',
    'test_keys.headers_match',
    'test_keys.title_match',
    'test_keys.accessible',
    'test_keys.blocking',
]

def explode(df, col):
    return df.explode(col, ignore_index=True)

def flatten(df, *args):
    _df = df
    
    args = list(args)
    
    if len(args) == 0:
        print("Case 0")
        return _df
    elif len(args) == 1:
        c1 = args.pop()
        if c1.endswith("[]"):
            _df = _df.explode(c1, ignore_index=True)
        
        print(f"Case 1: {c1}")
        return flatten(_df, args)
            
    # TODO: Continue here, cause this is kind of hard....
    elif len(args) >= 2:
        c1 = args.pop()
        c2 = args.pop()
        
        print(c1)
        print(c2)
        
        nc = c1 + "_" + c2
        if c1.endswith("[]"):
            _df = _df.explode(c1, ignore_index=True)
            nc = c1.replace("[]", "") + "_" + c2.replace("[]", "")
        
        print(f"Case 2: {nc}")
        return flatten(df, nc, *args)
    
            
for row in config[9:10]:
    cols = row.split(".")
    print(cols)
    
    if len(cols) == 0:
        continue
        
    col = cols.pop(0)
    df_test = flatten(df_test, col, cols)

df_test
    

['test_keys', 'network_events[]', 'address']
['network_events[]', 'address']
test_keys


TypeError: can only concatenate list (not "str") to list